## Part1: Inroduction

### 1.1: Installation
TensorFlow is an open-source machine learning framework developed by Google. It allows developers to build and train machine learning models using a data flow graph that represents computation. TensorFlow is versatile and widely used for various machine learning tasks, including computer vision and natural language processing. It also includes a high-level API called Keras for quick and easy model building and training.

In [ ]:
%pip install tensorflow
import tensorflow as tf

### 1.2 Tensors
In machine learning and TensorFlow, a tensor is a multi-dimensional array of numerical values that represents input data, intermediate computations, and output data in a neural network. Each tensor has a data type (e.g., float32 or int64) and a shape, which defines the number of dimensions and the size of each dimension. Tensors can be created using the `tf.constant()` function and manipulated using a wide range of mathematical operations. Tensors are a critical component of machine learning and TensorFlow, providing a flexible and efficient way to represent and manipulate data in a neural network.

#### 1.2.1 0-d Tensors

In [3]:
city = tf.constant("Beijing", tf.string)
pi = tf.cos(tf.constant(3.1415926, tf.float32))

print("`city` is a {}-d tensor".format(tf.rank(city).numpy()))
print("`pi` is a {}-d tensor".format(tf.rank(pi).numpy()))

`city` is a 0-d tensor
`pi` is a 0-d tensor


#### 1.2.2 1-d Tensors
Vectors and lists can create 1-d Tensors

In [6]:
cities = tf.constant(["Beijing", "Shanghai", "Guangzhou"], tf.string)
fvrt_numbers = tf.constant([3.1415926, 2.71828], tf.float32)

print("`cities` is a {}-d tensor with shape {}".format(tf.rank(cities).numpy(), tf.shape(cities)))
print("`fvrt_numbers` is a {}-d tensor with shape {}".format(tf.rank(fvrt_numbers).numpy(), tf.shape(fvrt_numbers)))

`cities` is a 1-d tensor with shape [3]
`fvrt_numbers` is a 1-d tensor with shape [2]


#### 1.2.3 Higher Dimensional Tensors

You can create 2-d, 3-d 4-d and even higher order Tensors

In [7]:
# a 2-d tensor
matrix = tf.constant([[1, 2, 3], [4, 5, 6]], tf.int32)

# a 3-d tensor
data = tf.ones([10, 256, 256], tf.int32)

# a 4-d tensor
images = tf.zeros([10, 256, 256, 3], tf.int32)

print("matrix is a {}-d tensor with shape {}".format(tf.rank(matrix).numpy(), tf.shape(matrix)))
print("data is a {}-d tensor with shape {}".format(tf.rank(data).numpy(), tf.shape(data)))
print("images is a {}-d tensor with shape {}".format(tf.rank(images).numpy(), tf.shape(images)))

matrix is a 2-d tensor with shape [2 3]
data is a 3-d tensor with shape [ 10 256 256]
images is a 4-d tensor with shape [ 10 256 256   3]


### 1.3 Computations on Tensors
In TensorFlow, computations occur in tensors through a process called data flow graph computation. A data flow graph is a directed graph where the nodes represent mathematical operations and the edges represent the tensors that flow between them. When a TensorFlow program is executed, it first builds a data flow graph, which specifies the operations to be performed and the order in which they should be executed.

Once the data flow graph is constructed, TensorFlow performs a computation called a forward pass, which propagates the input data through the graph to produce an output. During the forward pass, each node in the graph performs its operation on the input tensors it receives and passes the output tensor(s) to the next node(s) in the graph. This process continues until the output tensor(s) are produced by the final node in the graph.

After the forward pass is completed, TensorFlow performs a backward pass, which calculates the gradients of the output tensor(s) with respect to the input tensor(s). These gradients are used in a process called backpropagation, which updates the weights of the neural network to minimize the loss function.

Overall, computations in TensorFlow occur in tensors through a data flow graph computation process that involves a forward pass and a backward pass. This process allows TensorFlow to efficiently perform complex mathematical operations on large datasets and train sophisticated machine learning models.

In [8]:
# create nodes in a graph
a = tf.constant(15, name="a")
b = tf.constant(61, name="b")

# add them
c1 = tf.add(a, b, name="c1")
c2 = a + b # TensorFlow overrides the "+" operation so that it is able to act on tensors
print(c1)
print(c2)

tf.Tensor(76, shape=(), dtype=int32)
tf.Tensor(76, shape=(), dtype=int32)
